**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.
/home/naunauyoh/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/naunauyoh/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/naunauyoh/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/naunauyoh/anaconda3/lib/python3.7/site-pa

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        """ 
        Setting the value of the epsilon parameter
        """
        self.epsilon = e

    def act(self,s,train=True):
        """ 
        Returns the next action to do
        
        Returns:
        int: value between 0 and 4 (not included) with a random epsilon exploration
        """
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ 
        Acts via the policy of the agent, from a given state s
        """
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ T
        his function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ 
        Saves the information about the model
        """
        pass

    def load(self):
        """ 
        Loads a model from a file
        """
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The act function chooses an action to perform both during training with an $\epsilon$-greedy approahc and out of training. 

If it is in training mode, it will decided between two course of action based on the value epsilon. With probability $1 - \epsilon$, the function choose an action based on what it learned and the current state. Otherwise, with probability $\epsilon$ it chooses a random action.

Outside of training mode, it will choose the action based on what was learned previously and the current state.

The $\epsilon$ value is essential during the training mode as it forces the agent to explore unvisited states by sometimes choosing random action. Without a proper $\epsilon$ value the agent will either never explore high reward states if $\epsilon$ is too small or the ageng will never properly learn is $\epsilon$ is too high. 

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        """
        Saves a video of the agent in action
        """
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        """
        Helper function for a graphical representation of the grid
        """
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """
        Returns the new state, reward and whether the game ends
        """

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """Resets the game state and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [56]:
# parameters
size = 13
T = 200
temperature = 0.3
epochs_train = 100 # set small when debugging
epochs_test = 30 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array ```position``` is used to keep track of the position of the agent on the board. Note that initially all entries of the array are zeros except for the position of the agent with value $1$. Additionally, all the squares of the grid at distance 2 from the edges (the border) of the board have value $-1$. 

The array ```board``` is used to keep track of the reward given by each position on the grid. As the agent collects the rewards of its current position on the grid, the reward are reset to value $0$ after being collected. 


Note: I believe that there was a bug in the original code as we originally had:
```
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
```
instead of 

```
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
```

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        a = np.random.randint(0, self.n_action, size=1)[0]
        return a 

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(1, epochs + 1):
        
        ##### FILL IN HERE
        state = env.reset()
        
        game_over = False
        win = 0
        lose = 0
        
        while not game_over:
            a = agent.learned_act(state)
            state, r, game_over = env.act(a)
            
            if r > 0:
                win += r
            else:
                lose -= r
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. \t Average score ({})"
              .format(win, lose, score/e))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 11.5/14.0. 	 Average score (-2.5)
Win/lose count 9.0/21.0. 	 Average score (-7.25)
Win/lose count 9.5/9.0. 	 Average score (-4.666666666666667)
Win/lose count 11.5/14.0. 	 Average score (-4.125)
Win/lose count 3.0/8.0. 	 Average score (-4.3)
Win/lose count 13.0/13.0. 	 Average score (-3.5833333333333335)
Win/lose count 7.5/5.0. 	 Average score (-2.7142857142857144)
Win/lose count 7.0/9.0. 	 Average score (-2.625)
Win/lose count 11.0/15.0. 	 Average score (-2.7777777777777777)
Win/lose count 13.0/11.0. 	 Average score (-2.3)
Win/lose count 9.5/7.0. 	 Average score (-1.8636363636363635)
Win/lose count 13.5/16.0. 	 Average score (-1.9166666666666667)
Win/lose count 11.0/10.0. 	 Average score (-1.6923076923076923)
Win/lose count 10.5/12.0. 	 Average score (-1.6785714285714286)
Win/lose count 6.5/12.0. 	 Average score (-1.9333333333333333)
Win/lose count 9.0/17.0. 	 Average score (-2.3125)
Win/lose count 13.0/22.0. 	 Average score (-2.7058823529411766)
Win/lose count 7.0/19.0

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**Answer**:

We have that: 
$$
\begin{align*}
Q^\pi(s,a) &= E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
    &= r(s,a) + \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a] \\
    &= r(s,a) + \sum_{s', a'} p(s',a'|s,a) \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a,s_1=s', s_1=a'] \\
    &= r(s,a) + \sum_{s', a'} p(s',a'|s,a) \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_1=s', s_1=a'] \hbox{ ,by the Markovian property}\\
    &= r(s,a) + \sum_{s', a'} p(s',a'|s,a) \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_0=s', s_0=a'] \hbox{ ,by stationarity}\\
    &= r(s,a) + \sum_{s', a'} p(s',a'|s,a) \gamma Q^{\pi}(s',a')\\
    &= r(s,a) + E_{(s',a')\sim p(.|s,a)}[\gamma Q^{\pi}(s',a')]\\
    &= E_{(s',a')\sim p(.|s,a)}[r(s,a) + \gamma Q^{\pi}(s',a')]
\end{align*}
$$

Additionally:
$$
\begin{align*}
Q^*(s,a) &= \max_{\pi} Q^\pi(s,a) \\
    &= \max_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a) + \gamma Q^{\pi}(s',a')] \\
    &= r(s,a) + \gamma \max_{\pi} E_{(s',a')\sim p(.|s,a)}[Q^{\pi}(s',a')] \\
    &= r(s,a) + \gamma \max_{\pi} \sum_{s'} \sum_{a'} p(s'|s,a) \pi(a'|s') Q^{\pi}(s',a') \\
    &= r(s,a) + \gamma \sum_{s'}  p(s'|s,a) \max_{\pi} \sum_{a'} \pi(a'|s') Q^{\pi}(s',a') \\
    &= r(s,a) + \gamma \sum_{s'}  p(s'|s,a) \max_{\pi} \max_{a'} Q^{\pi}(s',a') \\
    &= r(s,a) + \gamma E_{s'\sim p(.|s,a)}[\max_{a'}Q^{*}(s',a')] \\
    &= E_{s'\sim p(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]
\end{align*}
$$


Finally, from the previous results we can define the following equation:
$$
E_{s'\sim p(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a') - Q^*(s,a)] = 0
$$

Note that minimizing 
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}
will help us to properly approximate a solution to the equation above. 

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()
        self.position = 0
        
    def remember(self, m):
        # Defines a cyclic memory
        if len(self.memory) < self.max_memory:
            self.memory.append(m)
        else:
            self.memory[self.position] = m
            self.position  = (self.position + 1) % self.max_memory
            
    def random_access(self):
        if len(self.memory) < 1:
            return None
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(1, epoch+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()

        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action
            prev_state = state
            # Get the next state, the reward and whether the game endse
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforce algorithm
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon=0.1, memory_size=100, batch_size=16, n_state=2):
        super(DQN, self).__init__(epsilon=epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        q_values = self.model.predict(s.reshape(-1, *s.shape)).flatten()
        
        return np.argmax(q_values)
    
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        # Get all batches
        batch = [self.memory.random_access() for i in range(self.batch_size)]
        
        # Get the necessary info
        input_states = np.stack([m[0] for m in batch])
        next_states = np.stack([m[1] for m in batch])
        actions = np.stack([m[2] for m in batch])
        rewards = np.stack([m[3] for m in batch])
        game_overs = np.stack([m[4] for m in batch])

        # Update Q value using the model
        target_q = self.model.predict(input_states)
        # Update Q value for the specific actions
        target_q[np.arange(self.batch_size), actions] = rewards + (1 - game_overs) * \
                self.discount * np.amax(self.model.predict(next_states))

        # HINT: Clip the target to avoid exploiding gradients. -- clipping is a bit tighter
        target_q = np.clip(target_q, -5, 5)

        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential([
            Flatten(input_shape=(5, 5, self.n_state)),
            Dense(32, activation="relu"),
            Dense(16, activation="relu"),
            Dense(4)
        ])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=1024, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train40.mp4'))







Epoch 001/100 | Loss 0.0193 | Win/lose count 2.5/4.0 (-1.5)
Epoch 002/100 | Loss 0.0072 | Win/lose count 4.5/6.0 (-1.5)
Epoch 003/100 | Loss 0.0832 | Win/lose count 3.0/4.0 (-1.0)
Epoch 004/100 | Loss 0.0738 | Win/lose count 1.5/2.0 (-0.5)
Epoch 005/100 | Loss 0.0049 | Win/lose count 2.0/7.0 (-5.0)
Epoch 006/100 | Loss 0.3935 | Win/lose count 3.5/1.0 (2.5)
Epoch 007/100 | Loss 0.0108 | Win/lose count 3.5/11.0 (-7.5)
Epoch 008/100 | Loss 0.0031 | Win/lose count 1.5/4.0 (-2.5)
Epoch 009/100 | Loss 0.0026 | Win/lose count 2.0/6.0 (-4.0)
Epoch 010/100 | Loss 0.0060 | Win/lose count 2.0/3.0 (-1.0)
Epoch 011/100 | Loss 0.0050 | Win/lose count 5.0/5.0 (0.0)
Epoch 012/100 | Loss 0.1632 | Win/lose count 6.5/4.0 (2.5)
Epoch 013/100 | Loss 0.0047 | Win/lose count 8.0/2.0 (6.0)
Epoch 014/100 | Loss 0.0057 | Win/lose count 1.5/5.0 (-3.5)
Epoch 015/100 | Loss 0.1911 | Win/lose count 6.5/11.0 (-4.5)
Epoch 016/100 | Loss 0.0051 | Win/lose count 2.5/3.0 (-0.5)
Epoch 017/100 | Loss 0.0086 | Win/lo

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential([
            Conv2D(32, kernel_size=(3, 3),activation='relu',
                 input_shape=(5, 5, self.n_state)),
            Conv2D(16, kernel_size=(2, 2),activation='relu'),
            Flatten(),
#            Dense(32, activation='relu'),
            Dense(4),
        ])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.05, epsilon = 0.2, memory_size=1024, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train40.mp4'))

Epoch 001/100 | Loss 0.0101 | Win/lose count 3.5/5.0 (-1.5)
Epoch 002/100 | Loss 0.1957 | Win/lose count 2.0/6.0 (-4.0)
Epoch 003/100 | Loss 0.0092 | Win/lose count 6.0/4.0 (2.0)
Epoch 004/100 | Loss 0.0079 | Win/lose count 4.5/5.0 (-0.5)
Epoch 005/100 | Loss 0.0077 | Win/lose count 5.0/5.0 (0.0)
Epoch 006/100 | Loss 0.0049 | Win/lose count 3.0/5.0 (-2.0)
Epoch 007/100 | Loss 0.0027 | Win/lose count 5.0/3.0 (2.0)
Epoch 008/100 | Loss 0.1710 | Win/lose count 4.0/7.0 (-3.0)
Epoch 009/100 | Loss 0.0063 | Win/lose count 8.0/3.0 (5.0)
Epoch 010/100 | Loss 0.0016 | Win/lose count 8.5/5.0 (3.5)
Epoch 011/100 | Loss 0.0046 | Win/lose count 11.0/3.0 (8.0)
Epoch 012/100 | Loss 0.0081 | Win/lose count 10.0/6.0 (4.0)
Epoch 013/100 | Loss 0.0024 | Win/lose count 9.5/4.0 (5.5)
Epoch 014/100 | Loss 0.0057 | Win/lose count 5.0/5.0 (0.0)
Epoch 015/100 | Loss 0.0112 | Win/lose count 3.0/3.0 (0.0)
Epoch 016/100 | Loss 0.0011 | Win/lose count 6.5/5.0 (1.5)
Epoch 017/100 | Loss 0.0106 | Win/lose count 8.0/

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, epsilon = 0.1, memory_size=2048, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, epsilon = 0.1, memory_size=2048, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 0.5/0.0. 	 Average score (0.5)
Win/lose count 0.5/0.0. 	 Average score (0.5)
Win/lose count 1.0/1.0. 	 Average score (0.3333333333333333)
Win/lose count 0/0.0. 	 Average score (0.25)
Win/lose count 0/0.0. 	 Average score (0.2)
Win/lose count 0.5/0.0. 	 Average score (0.25)
Win/lose count 0/0.0. 	 Average score (0.21428571428571427)
Win/lose count 1.5/0.0. 	 Average score (0.375)
Win/lose count 1.5/0.0. 	 Average score (0.5)
Win/lose count 3.0/0.0. 	 Average score (0.75)
Win/lose count 0/0.0. 	 Average score (0.6818181818181818)
Win/lose count 3.0/0.0. 	 Average score (0.875)
Win/lose count 1.5/0.0. 	 Average score (0.9230769230769231)
Win/lose count 1.5/0.0. 	 Average score (0.9642857142857143)
Win/lose count 0.5/0.0. 	 Average score (0.9333333333333333)
Win/lose count 1.0/0.0. 	 Average score (0.9375)
Win/lose count 2.5/1.0. 	 Average score (0.9705882352941176)
Win/lose count 0.5/0.0. 	 Average score (0.9444444444444444)
Win/lose count 2.5/0.0. 	 Average

In [16]:
HTML(display_videos('cnn_test10.mp4'))

In [17]:
HTML(display_videos('fc_test10.mp4'))

In [18]:
temperature_values = [0.1, 0.25, 0.5, 0.75, 0.9]
for temp in temperature_values:
    print("Temperature:", temp)
    env = Environment(grid_size=size, max_time=T, temperature=temp)
    test(agent_cnn,env,epochs_test,prefix=f'cnn_test_t{temp}_')
    test(agent_fc,env,epochs_test,prefix=f'fc_test_t{temp}_')
    print()
HTML("\n".join(display_videos(f'cnn_test_t{temp}_10.mp4') for temp in temperature_values))
HTML("\n".join(display_videos(f'fc_test_t{temp}_10.mp4') for temp in temperature_values))

Temperature: 0.1
Win/lose count 1.0/0.0. 	 Average score (1.0)
Win/lose count 0.5/0.0. 	 Average score (0.75)
Win/lose count 0/0.0. 	 Average score (0.5)
Win/lose count 0.5/0.0. 	 Average score (0.5)
Win/lose count 0/0.0. 	 Average score (0.4)
Win/lose count 0/0.0. 	 Average score (0.3333333333333333)
Win/lose count 2.0/0.0. 	 Average score (0.5714285714285714)
Win/lose count 1.0/0.0. 	 Average score (0.625)
Win/lose count 0/0.0. 	 Average score (0.5555555555555556)
Win/lose count 3.5/0.0. 	 Average score (0.85)
Win/lose count 1.0/0.0. 	 Average score (0.8636363636363636)
Win/lose count 0.5/0.0. 	 Average score (0.8333333333333334)
Win/lose count 0/0.0. 	 Average score (0.7692307692307693)
Win/lose count 0/0.0. 	 Average score (0.7142857142857143)
Win/lose count 0/0.0. 	 Average score (0.6666666666666666)
Win/lose count 0/0.0. 	 Average score (0.625)
Win/lose count 1.5/0.0. 	 Average score (0.6764705882352942)
Win/lose count 0/0.0. 	 Average score (0.6388888888888888)
Win/lose count 0.

Win/lose count 1.0/0.0. 	 Average score (1.2413793103448276)
Win/lose count 3.5/0.0. 	 Average score (1.3166666666666667)
Final score: 1.3166666666666667
Win/lose count 2.5/7.0. 	 Average score (-4.5)
Win/lose count 0/1.0. 	 Average score (-2.75)
Win/lose count 1.5/0.0. 	 Average score (-1.3333333333333333)
Win/lose count 1.0/2.0. 	 Average score (-1.25)
Win/lose count 1.5/1.0. 	 Average score (-0.9)
Win/lose count 2.5/4.0. 	 Average score (-1.0)
Win/lose count 1.5/5.0. 	 Average score (-1.3571428571428572)
Win/lose count 1.0/2.0. 	 Average score (-1.3125)
Win/lose count 0.5/0.0. 	 Average score (-1.1111111111111112)
Win/lose count 0.5/3.0. 	 Average score (-1.25)
Win/lose count 1.5/1.0. 	 Average score (-1.0909090909090908)
Win/lose count 0.5/0.0. 	 Average score (-0.9583333333333334)
Win/lose count 0/1.0. 	 Average score (-0.9615384615384616)
Win/lose count 2.5/1.0. 	 Average score (-0.7857142857142857)
Win/lose count 0.5/0.0. 	 Average score (-0.7)
Win/lose count 1.0/0.0. 	 Average 

Win/lose count 4.5/1.0. 	 Average score (2.3043478260869565)
Win/lose count 1.5/0.0. 	 Average score (2.2708333333333335)
Win/lose count 3.0/0.0. 	 Average score (2.3)
Win/lose count 1.0/0.0. 	 Average score (2.25)
Win/lose count 7.0/1.0. 	 Average score (2.388888888888889)
Win/lose count 6.0/1.0. 	 Average score (2.482142857142857)
Win/lose count 8.0/1.0. 	 Average score (2.6379310344827585)
Win/lose count 3.0/0.0. 	 Average score (2.65)
Final score: 2.65



**Answer**:

We note that both agents do not fully explore the environment. As we increase the temperature, there are more bonuses which helps to agent to explore more. We also not that, the CNN seems to fair better in the tests. This is most likely due to the ability of the CNN to capture local dependence across positions in the grid. 

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [19]:
def train_explore(agent, env, epoch, prefix='', epsilon_decrease=0.9):
    # Number of won games
    score = 0
    loss = 0

    for e in range(1, epoch + 1):
        agent.set_epsilon(agent.epsilon * epsilon_decrease)
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()

        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action on the environment
            prev_state = state
            # Get the next state, the reward and whether the game ends
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win += reward
            if reward < 0:
                lose -= reward

            # Apply the reinforce algorithm
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win - lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{:.1f} ({:.1f})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super(EnvironmentExploring, self).__init__(grid_size, max_time, temperature)
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
    
    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        
        if train:
            reward -= self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] += 0.1
        self.malus_position[self.x, self.y] = min(0.4, self.malus_position[self.x, self.y])
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                        self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over
    
    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    
## use those samples of code:
#In train explore:
#state, reward, game_over = env.act(action, train=True)

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
#reward = 0
#if train:
#    reward = -self.malus_position[self.x, self.y]
#self.malus_position[self.x, self.y] = 0.1

#reward = reward + self.board[self.x, self.y]
# 3 "feature" states instead of 2
#state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
#                                self.board.reshape(self.grid_size, self.grid_size,1),
#                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [20]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.3, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 001/100 | Loss 0.0179 | Win/lose count 7.0/57.5 (-50.5)
Epoch 002/100 | Loss 0.0069 | Win/lose count 7.0/55.6 (-48.6)
Epoch 003/100 | Loss 0.0116 | Win/lose count 4.5/60.0 (-55.5)
Epoch 004/100 | Loss 0.0082 | Win/lose count 5.0/62.0 (-57.0)
Epoch 005/100 | Loss 0.0092 | Win/lose count 11.0/36.9 (-25.9)
Epoch 006/100 | Loss 0.2209 | Win/lose count 10.5/51.3 (-40.8)
Epoch 007/100 | Loss 0.0237 | Win/lose count 4.0/54.7 (-50.7)
Epoch 008/100 | Loss 0.0059 | Win/lose count 10.0/46.1 (-36.1)
Epoch 009/100 | Loss 0.1777 | Win/lose count 13.5/34.9 (-21.4)
Epoch 010/100 | Loss 0.0077 | Win/lose count 11.0/41.4 (-30.4)
Epoch 011/100 | Loss 0.0219 | Win/lose count 12.0/39.0 (-27.0)
Epoch 012/100 | Loss 0.1952 | Win/lose count 15.0/32.9 (-17.9)
Epoch 013/100 | Loss 0.0055 | Win/lose count 19.0/26.9 (-7.9)
Epoch 014/100 | Loss 0.0084 | Win/lose count 17.5/39.4 (-21.9)
Epoch 015/100 | Loss 0.0071 | Win/lose count 18.5/28.0 (-9.5)
Epoch 016/100 | Loss 0.0045 | Win/lose count 22.5/26.1 (-3.6)


In [21]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 14.5/0.0. 	 Average score (14.5)
Win/lose count 21.0/0.0. 	 Average score (17.75)
Win/lose count 24.0/0.0. 	 Average score (19.833333333333332)
Win/lose count 2.0/0.0. 	 Average score (15.375)
Win/lose count 14.0/0.0. 	 Average score (15.1)
Win/lose count 20.5/1.0. 	 Average score (15.833333333333334)
Win/lose count 16.5/0.0. 	 Average score (15.928571428571429)
Win/lose count 12.5/0.0. 	 Average score (15.5)
Win/lose count 12.5/0.0. 	 Average score (15.166666666666666)
Win/lose count 9.5/0.0. 	 Average score (14.6)
Win/lose count 20.5/1.0. 	 Average score (15.045454545454545)
Win/lose count 10.0/1.0. 	 Average score (14.541666666666666)
Win/lose count 17.0/0.0. 	 Average score (14.73076923076923)
Win/lose count 10.0/0.0. 	 Average score (14.392857142857142)
Win/lose count 7.5/0.0. 	 Average score (13.933333333333334)
Win/lose count 16.0/0.0. 	 Average score (14.0625)
Win/lose count 9.5/1.0. 	 Average score (13.735294117647058)
Win/lose count 21.0/1.0. 	 Average score (1

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [22]:
def Generate_win_games(agent, env, epochs, memory_size=2000):
    """
    Generates winning games using the expert DQN
    
    Paramters:
    agent: Agent to use 
    env: Environment
    epochs (int): number of epochs
    memory_size (int): memory size
    
    Returns: 
    memory with tuples (state, next_state, action, reward, game_over) generated by the expert DQN
    """
    # memory of winning games
    memory = Memory(memory_size)
    
    # Number of won games
    score = 0
        
    for e in range(1, epochs+1):
        
        ##### FILL IN HERE
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
        
        while not game_over:
            # The agent performs an action in "test" mode (train = False)
            action = agent.act(state, train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
            
            m = [prev_state, state, action, reward, game_over]
            memory.remember(m)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

        # Update stats
        score = score + win - lose
        
        if e % 10 == 0:
            print("Game {}: Win/lose count {}/{}. Average score ({})"
                  .format(e, win, lose, score/(e)))
    
    return memory

In [57]:
class My_DQN(DQN):
    def __init__(self, grid_size, memory, epsilon=0.1, batch_size=16, n_state=2):
        super(My_DQN, self).__init__(grid_size=grid_size, epsilon=epsilon, 
                                     batch_size=batch_size, n_state=n_state)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = memory
        
        # Batch size when learning
        self.batch_size = batch_size

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool        
        input_states = np.zeros((self.batch_size, *s_.shape))
        target_q = np.zeros((self.batch_size, 4))
        
        # Get all batches
        batch = [self.memory.random_access() for i in range(self.batch_size)]
        
        # Get the necessary info
        input_states = np.stack([m[0] for m in batch])
        next_states = np.stack([m[1] for m in batch])
        actions = np.stack([m[2] for m in batch])
        rewards = np.stack([m[3] for m in batch])
        game_overs = np.stack([m[4] for m in batch])
        
        # Update Q value
        target_q = self.model.predict(input_states)
        target_q[np.arange(self.batch_size), actions] = rewards + (1 - game_overs) * self.discount * np.amax(
                self.model.predict(next_states))

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -5, 5)

        l = self.model.train_on_batch(input_states, target_q)
        return l

            
class My_DQN_CNN(My_DQN):
    def __init__(self, size, memory, epsilon, batch_size, n_state,lr=0.1,**kwargs):
        super(My_DQN_CNN, self).__init__(size, memory, epsilon, batch_size ,n_state,**kwargs)
        
        # design a CNN network with 2 conv layers of 16 and 32 filters and 1 fully connected layer
        
        ###### FILL IN
        model = Sequential([
            Conv2D(32, kernel_size=(3, 3),activation='relu',
                 input_shape=(5, 5, self.n_state)),
            Conv2D(16, kernel_size=(2, 2),activation='relu'),
            Flatten(),
#            Dense(32, activation='relu'),
            Dense(4),
        ])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [58]:
epochs = 500
memory_size = T * epochs
epsilon = 0.3
batch_size = 32
n_state = 3
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)

In [59]:
memory = Generate_win_games(agent, env, epochs, memory_size)

Game 10: Win/lose count 11.0/0. Average score (12.35)
Game 20: Win/lose count 20.0/0. Average score (14.425)
Game 30: Win/lose count 16.5/0. Average score (13.883333333333333)
Game 40: Win/lose count 2.0/1.0. Average score (13.475)
Game 50: Win/lose count 19.0/0. Average score (13.53)
Game 60: Win/lose count 16.0/0. Average score (13.858333333333333)
Game 70: Win/lose count 14.0/0. Average score (13.542857142857143)
Game 80: Win/lose count 13.0/0. Average score (13.51875)
Game 90: Win/lose count 3.0/0. Average score (13.2)
Game 100: Win/lose count 10.0/0. Average score (13.255)
Game 110: Win/lose count 15.5/0. Average score (13.113636363636363)
Game 120: Win/lose count 18.5/1.0. Average score (13.179166666666667)
Game 130: Win/lose count 13.5/2.0. Average score (13.234615384615385)
Game 140: Win/lose count 20.0/2.0. Average score (13.221428571428572)
Game 150: Win/lose count 14.0/1.0. Average score (13.213333333333333)
Game 160: Win/lose count 17.0/1.0. Average score (13.153125)
Game 1

In [60]:
# Training
mimic_agent = My_DQN_CNN(size, memory, epsilon, batch_size ,n_state, 0.1)
train(mimic_agent, env, epochs_train, prefix='mimic_cnn_train_explore')
HTML(display_videos('mimic_cnn_train_explore10.mp4'))

Epoch 001/100 | Loss 0.0175 | Win/lose count 5.5/9.0 (-3.5)
Epoch 002/100 | Loss 0.0018 | Win/lose count 12.0/9.0 (3.0)
Epoch 003/100 | Loss 0.0028 | Win/lose count 6.5/6.0 (0.5)
Epoch 004/100 | Loss 0.0024 | Win/lose count 12.0/10.0 (2.0)
Epoch 005/100 | Loss 0.0014 | Win/lose count 16.5/6.0 (10.5)
Epoch 006/100 | Loss 0.0012 | Win/lose count 15.5/7.0 (8.5)
Epoch 007/100 | Loss 0.0086 | Win/lose count 13.5/7.0 (6.5)
Epoch 008/100 | Loss 0.0011 | Win/lose count 18.5/6.0 (12.5)
Epoch 009/100 | Loss 0.1686 | Win/lose count 16.0/12.0 (4.0)
Epoch 010/100 | Loss 0.0087 | Win/lose count 8.5/5.0 (3.5)
Epoch 011/100 | Loss 0.1977 | Win/lose count 16.0/15.0 (1.0)
Epoch 012/100 | Loss 0.2768 | Win/lose count 9.5/5.0 (4.5)
Epoch 013/100 | Loss 0.0007 | Win/lose count 4.5/9.0 (-4.5)
Epoch 014/100 | Loss 0.0009 | Win/lose count 11.5/7.0 (4.5)
Epoch 015/100 | Loss 0.1909 | Win/lose count 11.0/8.0 (3.0)
Epoch 016/100 | Loss 0.0018 | Win/lose count 15.5/6.0 (9.5)
Epoch 017/100 | Loss 0.0089 | Win/lose

In [61]:
test(mimic_agent,env,epochs_test,prefix='mimic_cnn_test_explore')
HTML(display_videos('mimic_cnn_test_explore10.mp4'))

Win/lose count 4.0/0.0. 	 Average score (4.0)
Win/lose count 4.0/0.0. 	 Average score (4.0)
Win/lose count 3.5/1.0. 	 Average score (3.5)
Win/lose count 4.5/0.0. 	 Average score (3.75)
Win/lose count 2.0/0.0. 	 Average score (3.4)
Win/lose count 3.5/1.0. 	 Average score (3.25)
Win/lose count 2.5/0.0. 	 Average score (3.142857142857143)
Win/lose count 3.5/0.0. 	 Average score (3.1875)
Win/lose count 2.5/0.0. 	 Average score (3.111111111111111)
Win/lose count 2.5/0.0. 	 Average score (3.05)
Win/lose count 5.0/0.0. 	 Average score (3.227272727272727)
Win/lose count 2.5/0.0. 	 Average score (3.1666666666666665)
Win/lose count 5.5/0.0. 	 Average score (3.3461538461538463)
Win/lose count 3.5/0.0. 	 Average score (3.357142857142857)
Win/lose count 2.5/0.0. 	 Average score (3.3)
Win/lose count 3.0/0.0. 	 Average score (3.28125)
Win/lose count 4.5/1.0. 	 Average score (3.2941176470588234)
Win/lose count 3.5/0.0. 	 Average score (3.3055555555555554)
Win/lose count 2.0/1.0. 	 Average score (3.184

Our attempt to mimic the expert DQN did not fair well. Indeed, we are trying to copy an existing model by looking at its experiences. This narrows the size of the set information (state, next_state, actions, rewards, game_overs) that the model can see. This prevents proper exploration which is one reason why our mimic model did not fair well. However, it performs better than the basic FC and CNN model that do not use ```train_explore```. 